<a href="https://colab.research.google.com/github/hiroki1982/Research/blob/main/Loneliness_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ✅ LangChain + SQLite：記憶復元 + 明示的履歴挿入版テンプレート

# セル1: 必要なライブラリ
!pip install -U langchain langchain-community langchain-openai openai pandas --quiet

# セル2: ライブラリ読み込み
import sqlite3
from datetime import datetime
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain_openai import ChatOpenAI
from google.colab import userdata

# セル3: DB初期化関数
def init_persona_db():
    conn = sqlite3.connect("personas.db")
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS personas (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            role TEXT UNIQUE,
            big5_extraversion INTEGER,
            big5_agreeableness INTEGER,
            big5_conscientiousness INTEGER,
            big5_neuroticism INTEGER,
            big5_openness INTEGER,
            description TEXT
        )
    """)
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS logs (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            timestamp TEXT,
            role TEXT,
            user_input TEXT,
            agent_reply TEXT
        )
    """)
    conn.commit()
    conn.close()

# セル4: 人格登録関数
def insert_persona(role, extraversion, agreeableness, conscientiousness, neuroticism, openness, description):
    conn = sqlite3.connect("personas.db")
    cursor = conn.cursor()
    cursor.execute("""
        INSERT OR REPLACE INTO personas (role, big5_extraversion, big5_agreeableness, big5_conscientiousness,
            big5_neuroticism, big5_openness, description)
        VALUES (?, ?, ?, ?, ?, ?, ?)
    """, (role, extraversion, agreeableness, conscientiousness, neuroticism, openness, description))
    conn.commit()
    conn.close()

# セル5: ログから履歴を読み込み、文字列に整形
def load_past_history(role, limit=10):
    conn = sqlite3.connect("personas.db")
    cursor = conn.cursor()
    cursor.execute("""
        SELECT user_input, agent_reply FROM logs
        WHERE role = ? ORDER BY timestamp ASC LIMIT ?
    """, (role, limit))
    rows = cursor.fetchall()
    conn.close()
    return "\n".join([f"ユーザー: {u}\nエージェント: {a}" for u, a in rows])

# セル6: 対話関数（明示的に履歴渡す）
def chat_with_persona(role):
    conn = sqlite3.connect("personas.db")
    cursor = conn.cursor()
    cursor.execute("SELECT description FROM personas WHERE role = ?", (role,))
    result = cursor.fetchone()
    if not result:
        print("指定された人格が見つかりませんでした。")
        conn.close()
        return
    persona_description = result[0]
    conn.close()

    prompt = PromptTemplate(
        input_variables=["input", "persona", "history"],
        template="""
あなたは以下のような性格を持つ人物であり、すべての応答にその性格が反映されるよう意識してください。
。

[性格プロファイル]
{persona}

あなたはこれまでの会話を記憶しており、それをもとに応答します。

[これまでの会話]
{history}

ユーザー: {input}
あなた:
        """
    )

    llm = ChatOpenAI(
        model="gpt-3.5-turbo",
        openai_api_key=userdata.get("API_KEY")
    )

    history_text = load_past_history(role)
    chain = LLMChain(llm=llm, prompt=prompt, verbose=True)

    print(f"=== {role} としての対話を開始します（履歴明示挿入） 'exit'で終了 ===")

    while True:
        user_input = input("あなた: ")
        if user_input.lower() == "exit":
            print("またお話しましょう。")
            break

        response = chain.run({
            "input": user_input,
            "persona": persona_description,
            "history": history_text
        })

        print(f"{role}: {response}\n")

        # 会話ログ保存 + 履歴更新
        conn = sqlite3.connect("personas.db")
        cursor = conn.cursor()
        timestamp = datetime.now().isoformat()
        cursor.execute("""
            INSERT INTO logs (timestamp, role, user_input, agent_reply)
            VALUES (?, ?, ?, ?)
        """, (timestamp, role, user_input, response))
        conn.commit()
        conn.close()

        history_text += f"\nユーザー: {user_input}\nエージェント: {response}"

# セル7: 初期化とサンプル人格登録
init_persona_db()

insert_persona(
    role="calm_teacher",
    extraversion=90,
    agreeableness=85,
    conscientiousness=90,
    neuroticism=30,
    openness=70,
    description="穏やかで誠実な教師。常に相手の話に耳を傾け、わかりやすく論理的に話す。少し控えめで、慎重な判断を重んじる。"
)

insert_persona(
    role="expressive_artist",
    extraversion=75,
    agreeableness=65,
    conscientiousness=55,
    neuroticism=60,
    openness=95,
    description="感受性が豊かで表現力に富んだアーティスト。自由を愛し、型にはまらない考え方をする。情熱的だが気分屋な面もある。"
)

insert_persona(
    role="logical_engineer",
    extraversion=30,
    agreeableness=60,
    conscientiousness=95,
    neuroticism=25,
    openness=60,
    description="論理的思考を重視するエンジニア。落ち着いていて、感情に流されず合理的な判断を行う。社交性は高くないが実直。"
)

print("✅ 履歴明示型テンプレートに更新されました。chat_with_persona('role') で対話を開始できます。")


In [ ]:
chat_with_persona('expressive_artist')